# 🚀 Deploy Final de Modelos de Séries Temporais - CVC Lojas

## 🎯 Objetivo
Treinar a versão **FINAL** do modelo de previsão de vendas, utilizando os melhores hiperparâmetros validados, e registrar um artefato "All-in-One" no Unity Catalog pronto para inferência produtiva.

## 📅 Escopo Temporal (Concept Drift Prevention)
**REGRA DE NEGÓCIO CRÍTICA:** Para evitar dados obsoletos (pré-pandemia/2019) que não refletem o comportamento atual do consumidor, este treino utiliza estritamente:
*   **Início:** `2021-01-01`
*   **Fim:** `2025-12-31`

Qualquer dado fora deste intervalo é filtrado no Spark antes da injeção.

## 📦 UnifiedForecaster (Model Wrapper)
O modelo é salvo como um MLflow PyFunc customizado (`UnifiedForecaster`) que encapsula:
1.  **Pipeline de Preprocessamento** (Scaler, Missing Values) - Garante que o input bruto seja tratado igual ao treino.
2.  **Modelo Darts Treinado** (LGBM, TFT, etc).
3.  **Lógica de Pós-processamento** (Inverse Transform) - Entrega a previsão na escala real (R$).


In [0]:
# --- CONFIGURAÇÕES DE AMBIENTE ---
%load_ext autoreload
%autoreload 2

import sys
import os
# Garante que o src seja encontrável
sys.path.append(os.getcwd())

# Imports da nossa lib modularizada
from src.validation.config import Config
from src.validation.data import DataIngestion
from src.validation.pipeline import ProjectPipeline
from src.deploy.wrapper import UnifiedForecaster

# Bibliotecas Externas
import mlflow
import pandas as pd
import pickle
import pyspark.sql.functions as F
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema, ColSpec
from darts.models import LightGBMModel, TFTModel # Exemplo: Importe o modelo vencedor aqui

# --- CONFIGS GLOBAIS ---
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

In [0]:
# --- 1. DEFINIÇÃO DO ESCOPO TEMPORAL (HARDCODED) ---
# Sobrescrevemos configs de widget para garantir a regra de negócio de Deploy
config = Config(spark)
config.DATA_START = "2021-01-01"
config.TRAIN_END_DATE = "2025-12-31"
config.INGESTION_END = "2025-12-31" # Para deploy, usamos tudo até hoje
config.SCHEMA="cvc_pred"

print(f"⏱️ PERÍODO DE TREINO DEFINIDO: {config.DATA_START} até {config.TRAIN_END_DATE}")
print(f"⚠️ Dados anteriores a 2021 serão ignorados (Concept Drift Strategy).")

In [0]:
# --- 2. INGESTÃO E FILTRAGEM (SPARK SIDE) ---
ingestion = DataIngestion(spark, config)

# a) Cria o dataset bruto via Feature Store
df_spark_raw = ingestion.create_training_set()

# b) FILTRAGEM RGIDA NO SPARK (Eficiente)
print(f"   ♻️ Aplicando filtro temporal no Cluster Spark ({config.DATA_START} - {config.TRAIN_END_DATE})...")
df_spark_filtered = df_spark_raw.filter(
    F.col("DATA").between(config.DATA_START, config.TRAIN_END_DATE)
)

# c) Carrega suportes globais (já filtrados pela classe DataIngestion se config estiver certa, mas reforçamos)
df_global_support = ingestion.get_global_support()
df_global_support = df_global_support[config.DATA_START : config.TRAIN_END_DATE]

# d) Converte para Darts (Pandas no Driver)
target_series_list, full_covariates_list = ingestion.build_darts_objects(df_spark_filtered, df_global_support)

print(f"\n✅ Dataset Final Pronto para Treino: {len(target_series_list)} Lojas.")

In [0]:
# --- 3. PIPELINE E TREINAMENTO FINAL ---

# Inicializa Pipeline Unificado
pipeline = ProjectPipeline()

print("⚙️ Fit & Transform dos Scalers...")
# Ajusta scalers nos dados filtrados (2021-2025)
pipeline.fit(target_series_list, full_covariates_list)
scaled_series, scaled_covariates = pipeline.transform(target_series_list, full_covariates_list)

# Inicializa o Modelo Vencedor (Exemplo: LightGBM com params otimizados)
print("?? Iniciando Treinamento do Modelo (LightGBM)...")
model = LightGBMModel(
    lags=12, 
    lags_future_covariates=[0,1,2,3],
    output_chunk_length=1,
    random_state=42
)

# Treinamento Full
model.fit(
    scaled_series,
    future_covariates=scaled_covariates
)
print("✅ Modelo Treinado com Sucesso!")

In [0]:
# --- 4. REGISTRO NO UNITY CATALOG (ALL-IN-ONE) ---

mlflow.set_experiment(config.EXPERIMENT_NAME)

# Nome do Modelo no Catálogo
catalog_model_name = f"{config.CATALOG}.{config.SCHEMA}.cvc_lojas_forecast_production"

with mlflow.start_run(run_name="Deploy_Production_2025") as run:
    # Salvando artefatos auxiliares para o wrapper
    pipeline_path = "pipeline.pkl"
    model_path = "lgbm_model.pkl"
    covariates_path = "future_covariates.pkl"
    
    with open(pipeline_path, "wb") as f: pickle.dump(pipeline, f)
    with open(model_path, "wb") as f: pickle.dump(model, f)
    # Salvamos as covariáveis globais scaladas para uso futuro se necessário (opcional)
    with open(covariates_path, "wb") as f: pickle.dump(scaled_covariates, f)
    
    artifacts = {
        "pipeline": pipeline_path,
        "darts_model": model_path,
        "future_covariates": covariates_path
    }
    
    # Assinatura (Simples)
    input_schema = Schema([ColSpec("string", "DATA"), ColSpec("string", "CODIGO_LOJA")]) 
    output_schema = Schema([ColSpec("double", "PREVISAO_VENDA")])
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)
    
    print(f"💾 Registrando modelo UnifiedForecaster em: {catalog_model_name}...")
    
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=UnifiedForecaster(), # Nossa classe wrapper customizada
        artifacts=artifacts,
        pip_requirements=["darts", "pandas", "numpy", "lightgbm"],
        signature=signature,
        registered_model_name=catalog_model_name
    )
    
    print(f"✨ Sucesso! Modelo registrado e pronto para inferência: {catalog_model_name}")

# Limpeza local
for p in [pipeline_path, model_path, covariates_path]:
    if os.path.exists(p): os.remove(p)